# Some ways to use the Responses API in Llama Stack

The development of this notebook was assisted by Cursor using Claude 4 Sonnet.

Before getting started, install and run the Llama Stack server, e.g.,

```
pip install llama-stack
llama stack run run.yaml --image-type venv
```

And also run the National Parks Service Model Context Protocol (MCP) server:

```
python nps_mcp_server.py --transport sse --port 3005
```

See [README_NPS](../README_NPS.md) for more information about this server.

## Configuration

Here we point to the locations of the servers we just started up above.  Also, we provide the model ID for the model we want to use.  The model ID should be one that that is specified in [run.yaml](./run.yaml).  In the [run.yaml](./run.yaml) included here, we have the following models defined:

- `gpt-3.5-turbo` and `gpt4o` are models from OpenAI.  They will only work if you have OPENAI_API_KEY set in your environment to a [valid OpenAI API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key).
- `Llama-3.3-70B-Instruct` is a a model from Meta Llama.  This will only work if you have an API key for the hosted [Llama API](https://www.llama.com/products/llama-api/).
- `meta-llama/llama-3-3-70b-instruct` is the same model running on watsonx.ai (which has a somewhat different style for model IDs).  This will only work if you have a WATSONX_API_KEY and WATSONX_PROJECT_ID set (which requires an IBM Cloud account).  You may also need to set WATSONX_BASE_URL set if your watsonx.ai instance is running anywhere other than US South (which is the default).

If you can't or don't want to get any of those API keys, you can update [run.yaml](./run.yaml) to use [another inference provider](https://llama-stack.readthedocs.io/en/latest/providers/inference/index.html#overview).  Llama Stack includes numerous providers for calling hosted models like the ones above.  It also includes providers to call models that you deploy and run yourself using a model serving capability, e.g., the [vLLM provider](https://llama-stack.readthedocs.io/en/latest/providers/inference/remote_vllm.html) or the [ollama provider](https://llama-stack.readthedocs.io/en/latest/providers/inference/remote_ollama.html).

In [1]:
LLAMA_STACK_URL = "http://localhost:8321/"
NPS_MCP_URL = "http://localhost:3005/sse/"
LLAMA_STACK_MODEL_ID = "gpt-4o"

## Using the Llama Stack client

The most obvious way to use the Responses API in Llama Stack is via the Llama Stack client.  That's the way that is most seamlessly integrated with Llama Stack itself, so we would recommend it for most beginning users who don't already have a commitment to another client library.  Here are some examples using this approach.

In [27]:
from llama_stack_client import LlamaStackClient

client = LlamaStackClient(base_url="http://localhost:8321")

We will start with a trivial example:

In [3]:
simple_llama_stack_client_response = client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="What is the capital of France?"
)

simple_llama_stack_client_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ResponseObject(id='resp-9695bb40-c676-4077-a16a-fd5f032428dd', created_at=1754688405, model='gpt-4o', object='response', output=[OutputOpenAIResponseMessage(content=[OutputOpenAIResponseMessageContentUnionMember2(text='The capital of France is Paris.', type='output_text', annotations=[])], role='assistant', type='message', id='msg_37a6cfb0-c2ec-4d09-9862-c647badbaeaf', status='completed')], parallel_tool_calls=False, status='completed', text=Text(format=TextFormat(type='text', description=None, name=None, schema_=None, strict=None)), error=None, previous_response_id=None, temperature=None, top_p=None, truncation=None, user=None)

The response object is a little complex to read, so we provide a function to print it out in a more readable format:

In [4]:
def print_simple_response(response):
    print(f"ID: {response.id}")
    print(f"Status: {response.status}")
    print(f"Model: {response.model}")
    print(f"Created at: {response.created_at}")
    print(f"Output type: {response.output[0].type}")
    print(f"Response content: {response.output[0].content[0].text}")

In [5]:
print_simple_response(simple_llama_stack_client_response)

ID: resp-9695bb40-c676-4077-a16a-fd5f032428dd
Status: completed
Model: gpt-4o
Created at: 1754688405
Output type: message
Response content: The capital of France is Paris.


Now we will move on to a more substantial example using the NPS MCP server you set up at the top of this notebook.

In [43]:
complex_llama_stack_client_response = client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="Tell me about some events at some parks in Rhode Island.",
    tools=[
        {
            "type": "mcp",
            "server_url": NPS_MCP_URL,
            "server_label": "National Parks Service tools",
        }
    ]
)

complex_llama_stack_client_response

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ResponseObject(id='resp-f1caa1ba-51bd-4cfd-831a-a802e08d45e0', created_at=1754745244, model='gpt-4o', object='response', output=[OutputOpenAIResponseOutputMessageMcpListTools(id='mcp_list_824083e6-e7c6-4f21-83e2-94481e734641', server_label='National Parks Service tools', tools=[OutputOpenAIResponseOutputMessageMcpListToolsTool(input_schema={'type': 'object', 'properties': {'state_code': {'type': 'string', 'description': ''}, 'park_code': {'type': 'string', 'description': ''}, 'query': {'type': 'string', 'description': ''}, 'limit': {'type': 'integer', 'description': ''}}, 'required': ['state_code', 'park_code', 'query', 'limit']}, name='search_parks', description="Search for national parks by state, park code, or query string.\n\nArgs:\n    state_code: Two-letter state code (e.g., 'CA', 'NY')\n    park_code: Four-letter park code (e.g., 'yell', 'acad')\n    query: Search query for park names or descriptions\n    limit: Maximum number of results to return (default: 10)\n\nReturns:\n    

The output for this one is more complicated, so we'll need to make a more powerful print method for it:

In [7]:
def print_response(response):
    print(f"ID: {response.id}")
    print(f"Status: {response.status}")
    print(f"Model: {response.model}")
    print(f"Created at: {response.created_at}")
    print(f"Output items: {len(response.output)}")
    
    for i, output_item in enumerate(response.output):
        if len(response.output) > 1:
            print(f"\n--- Output Item {i+1} ---")
        print(f"Output type: {output_item.type}")
        
        if output_item.type in ("text", "message"):
            print(f"Response content: {output_item.content[0].text}")
        elif output_item.type == "mcp_list_tools":
            print_mcp_list_tools(output_item)
        elif output_item.type == "mcp_call":
            print_mcp_call(output_item)
        else:
            print(f"Response content: {output_item.content}")

def print_mcp_call(mcp_call):
    """Print MCP call in a nicely formatted way"""
    print(f"\n🛠️  MCP Tool Call: {mcp_call.name}")
    print(f"   Server: {mcp_call.server_label}")
    print(f"   ID: {mcp_call.id}")
    print(f"   Arguments: {mcp_call.arguments}")
    
    if mcp_call.error:
        print("   ❌ Error: {mcp_call.error}")
    elif mcp_call.output:
        print("   ✅ Output:")
        # Try to format JSON output nicely
        try:
            import json
            parsed_output = json.loads(mcp_call.output)
            print(json.dumps(parsed_output, indent=4))
        except:
            # If not valid JSON, print as-is
            print(f"   {mcp_call.output}")
    else:
        print("   ⏳ No output yet")

def print_mcp_list_tools(mcp_list_tools):
    """Print MCP list tools in a nicely formatted way"""
    print(f"\n🔧 MCP Server: {mcp_list_tools.server_label}")
    print(f"   ID: {mcp_list_tools.id}")
    print(f"   Available Tools: {len(mcp_list_tools.tools)}")
    print("=" * 80)
    
    for i, tool in enumerate(mcp_list_tools.tools, 1):
        print(f"\n{i}. {tool.name}")
        print(f"   Description: {tool.description}")
        
        # Parse and display input schema
        schema = tool.input_schema
        if schema and 'properties' in schema:
            properties = schema['properties']
            required = schema.get('required', [])
            
            print("   Parameters:")
            for param_name, param_info in properties.items():
                param_type = param_info.get('type', 'unknown')
                param_desc = param_info.get('description', 'No description')
                required_marker = " (required)" if param_name in required else " (optional)"
                print(f"     • {param_name} ({param_type}){required_marker}")
                if param_desc:
                    print(f"       {param_desc}")
        
        if i < len(mcp_list_tools.tools):
            print("-" * 40)

In [29]:
print_response(complex_llama_stack_client_response)

ID: resp-87ecbcd8-61b7-4bce-a537-2a7f7086687a
Status: completed
Model: gpt-4o
Created at: 1754688674
Output items: 7

--- Output Item 1 ---
Output type: mcp_list_tools

🔧 MCP Server: National Parks Service tools
   ID: mcp_list_4230fcf2-111d-4432-90d0-3d9321af3877
   Available Tools: 5

1. search_parks
   Description: Search for national parks by state, park code, or query string.

Args:
    state_code: Two-letter state code (e.g., 'CA', 'NY')
    park_code: Four-letter park code (e.g., 'yell', 'acad')
    query: Search query for park names or descriptions
    limit: Maximum number of results to return (default: 10)

Returns:
    JSON string with park information including name, description, website, and location
   Parameters:
     • state_code (string) (required)
     • park_code (string) (required)
     • query (string) (required)
     • limit (integer) (required)
----------------------------------------

2. get_park_alerts
   Description: Get current alerts for a specific national 

You can see the full output from the call to the Responses API above asking "Tell me about some events at some parks in Rhode Island.".  Here are some of the highlights:

* Output Item 1 shows that it called the NPS server to list all of that server's tools.
* Output Item 2 shows that it then called the `search_parks` tool from that server with arguments `{"state_code":"RI","limit":5}` and got back a list of 4 national parks in Rhode Island.  Since the limit it specified was 5 and it only got 4, presumably that's all there are in that state.
* Output Item 3 shows that it called `get_park_events` with arguments `{"park_code": "blrv", "limit": 5}`.  Notice that the value for `park_code` is the same as the value for `code` in the first output from the `search_parks` tool.  So the model recognized that the `code` field in the output of `search_parks` corresponds to the `park_code` field in the input of `get_park_events`.  It gets back a list of events for that park.
* Output Items 4-6 also call `get_park_events` with the other three park codes that had been returned by the `search_parks` tool.
* Output Item 7 then provides the actual response to be sent to the user.  It describes the outputs of all four of the calls to `get_park_events` in a human-friendly form.

One important thing to note is that all 7 output items came from *one* call to the Responses API.  This illustrates one of the advantages of this API and its implementation in Llama Stack: Llama Stack handles all of the coordination between all of these steps and calls the model and to the MCP server.  You could accomplish the same thing using basic "completions" API that allow you to specify tools, but it would then be up to you in the client to do all that coordinating.

## Using the OpenAI client

In the examples above we used the Llama Stack client to connect to Llama Stack.  However, since we're calling an OpenAI-compatible API (Responses), we can also use the OpenAI client to do the same thing.  The setup is a little bit clunkier than it is using the Llama Stack client.  With the OpenAI client, you have to add some extra stuff at the end of the URL to point the client at the OpenAI-compatible part of the overall Llama Stack.  However, once you do that, you can do the same things with the OpenAI client as we did with the Llama Stack client above.

Using the OpenAI client may be the best fit if you are writing code for a project that already uses that client for other purposes and you don't want to add any more dependencies to that project.

We start with the slightly clunky setup:

In [9]:

from openai import OpenAI

# Direct OpenAI client instantiation with Llama Stack
openai_client = OpenAI(
    api_key="no-key-needed",  # Llama Stack typically doesn't require a real key
    base_url=LLAMA_STACK_URL + "v1/openai/v1", # This suffix gets to the part of the URL that is specific to the OpenAI API, which you need here since you are using the OpenAI client.
)

Notice that while this is the OpenAI client, the URL we pointed it to is our Llama Stack server.  So even though we're using a different client here, we're still using the same server to handle the request.  Here we see that the simple example we used above works the same way as it did with the Llama Stack client:

In [10]:

simple_openai_client_response = openai_client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="What is the capital of France?"
)

print_simple_response(simple_openai_client_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


ID: resp-f043f493-db7e-4445-a2b5-cd085aaaa83e
Status: completed
Model: gpt-4o
Created at: 1754688419.0
Output type: message
Response content: The capital of France is Paris.


And similarly, the complex example using the NPS MCP server also works the same way as it did with the Llama Stack client:

In [11]:
complex_openai_client_response = openai_client.responses.create(
    model=LLAMA_STACK_MODEL_ID,
    input="Tell me about some events at some parks in Rhode Island.",
    tools=[
        {
            "type": "mcp",
            "server_url": NPS_MCP_URL,
            "server_label": "National Parks Service tools",
        }
    ]
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


Because both the OpenAI client and the Llama Stack client are wrapping the same underlying server API, the structures of their outputs wind up being the same too, so the complicated print_response method that prints out the response objects for the Llama Stack client also prints out the response objects for the OpenAI client.

In [12]:
print_response(complex_openai_client_response)


ID: resp-8060ed83-452f-41ad-9ef7-05b77d3a0b58
Status: completed
Model: gpt-4o
Created at: 1754688419.0
Output items: 6

--- Output Item 1 ---
Output type: mcp_list_tools

🔧 MCP Server: National Parks Service tools
   ID: mcp_list_f69d1745-69fb-43e8-8116-5284341e4cc8
   Available Tools: 5

1. search_parks
   Description: Search for national parks by state, park code, or query string.

Args:
    state_code: Two-letter state code (e.g., 'CA', 'NY')
    park_code: Four-letter park code (e.g., 'yell', 'acad')
    query: Search query for park names or descriptions
    limit: Maximum number of results to return (default: 10)

Returns:
    JSON string with park information including name, description, website, and location
   Parameters:
     • state_code (string) (required)
     • park_code (string) (required)
     • query (string) (required)
     • limit (integer) (required)
----------------------------------------

2. get_park_alerts
   Description: Get current alerts for a specific nationa

As you can see, we get the same results here that we did with the Llama Stack client, as you would expect since we're calling the same Llama Stack server with the same arguments.

## LangChain

Both of the examples above are using clients that directly mirror the Responses API.  That means you have complete control over and visibility into exactly what's going into the API and exactly what's coming out of the API.  That can be useful and powerful, but it can also be a little complicated and tedious.  A lot of users prefer to use a framework that provides higher level abstractions.  Examples of such frameworks include LangChain, LangGraph, LlamaIndex, Haystack, and many more.

Here we configure LangChain with `use_responses_api` so it will use the same Responses API that we use above.  However, it hides a lot of the details of that API making the code simpler.

In [20]:
from langchain_openai import ChatOpenAI

# Pointing to your local Responses API server
llm = ChatOpenAI(
    model=LLAMA_STACK_MODEL_ID,
    base_url=LLAMA_STACK_URL + "v1/openai/v1", # This suffix gets to the part of the URL that is specific to the OpenAI API, which you need here since you are using the OpenAI client.,
    use_responses_api=True
)

In [21]:
langchain_simple_response = llm.invoke("What is the capital of France?")
print(langchain_simple_response.content)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


[{'type': 'text', 'text': 'The capital of France is Paris.', 'annotations': []}]


Now let's create a LangChain agent that uses the existing `llm` variable and connects to the NPS MCP server for tool use. This demonstrates how to integrate MCP-based tools directly into a LangChain agent workflow.


In [ ]:
from os import NGROUPS_MAX
from langchain_mcp_adapters.client import MultiServerMCPClient

langchain_client = MultiServerMCPClient(
    {
        "nps": {
            "url": NPS_MCP_URL,
            "transport": "sse",
        }
    }
)
tools = await langchain_client.get_tools()
tools

INFO:httpx:HTTP Request: GET http://localhost:3005/sse/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3005/messages/?session_id=861653313157445ba2dc37eb20fbdc88 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3005/messages/?session_id=861653313157445ba2dc37eb20fbdc88 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3005/messages/?session_id=861653313157445ba2dc37eb20fbdc88 "HTTP/1.1 202 Accepted"


[StructuredTool(name='search_parks', description="Search for national parks by state, park code, or query string.\n\nArgs:\n    state_code: Two-letter state code (e.g., 'CA', 'NY')\n    park_code: Four-letter park code (e.g., 'yell', 'acad')\n    query: Search query for park names or descriptions\n    limit: Maximum number of results to return (default: 10)\n\nReturns:\n    JSON string with park information including name, description, website, and location", args_schema={'properties': {'state_code': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'State Code'}, 'park_code': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Park Code'}, 'query': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Query'}, 'limit': {'default': 10, 'title': 'Limit', 'type': 'integer'}}, 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11b

In [44]:
await tools[0].arun({"state_code": "CA", "limit": 10})

INFO:httpx:HTTP Request: GET http://localhost:3005/sse/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3005/messages/?session_id=cfcf572b69b041f1865377762ca5f1dc "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3005/messages/?session_id=cfcf572b69b041f1865377762ca5f1dc "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3005/messages/?session_id=cfcf572b69b041f1865377762ca5f1dc "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3005/messages/?session_id=cfcf572b69b041f1865377762ca5f1dc "HTTP/1.1 202 Accepted"


'{\n  "total": "34",\n  "parks": [\n    {\n      "name": "Alcatraz Island",\n      "code": "alca",\n      "description": "Alcatraz reveals stories of American incarceration, justice, and our common humanity. This small island was once a fort, a military prison, and a maximum security federal penitentiary. In 1969, the Indians of All Tribes occupied Alcatraz for 19 months in the name of freedom and Native American civil rights. We invite you to explore Alcatraz\'s complex history and natural beauty.",\n      "website": "https://www.nps.gov/alca/index.htm",\n      "states": "CA",\n      "designation": "",\n      "latitude": "37.82676234",\n      "longitude": "-122.4230206"\n    },\n    {\n      "name": "Butterfield Overland National Historic Trail",\n      "code": "buov",\n      "description": "In 1857, businessman and transportation entrepreneur John Butterfield was awarded a contract to establish an overland mail route between the eastern United States and growing populations in the Fa

## LangGraph

In [ ]:
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(llm, tools)

In [35]:
langchain_complex_response = agent.invoke({"messages": "Tell me about some events at some parks in Rhode Island."})

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/openai/v1/responses "HTTP/1.1 200 OK"


In [32]:
langchain_complex_response

{'messages': [HumanMessage(content='Tell me about some events at some parks in Rhode Island.', additional_kwargs={}, response_metadata={}, id='68fa3df2-1a5f-404d-8451-dd379c9bf9d4'),
  AIMessage(content=[], additional_kwargs={'__openai_function_call_ids__': {'call_RQvXyoNDa3CxWF5NqLueiTl0': 'fc_033728c6-a8a6-4629-bd68-8b12d46e1136'}}, response_metadata={'id': 'resp-f6a53da4-652b-47ad-b6b0-da70553a2244', 'created_at': 1754688728.0, 'model': 'gpt-4o', 'object': 'response', 'status': 'completed', 'model_name': 'gpt-4o'}, id='resp-f6a53da4-652b-47ad-b6b0-da70553a2244', tool_calls=[{'name': 'search_parks', 'args': {'state_code': 'RI', 'limit': 5}, 'id': 'call_RQvXyoNDa3CxWF5NqLueiTl0', 'type': 'tool_call'}]),
  ToolMessage(content="Error: NotImplementedError('StructuredTool does not support sync invocation.')\n Please fix your mistakes.", name='search_parks', id='0d0aefd5-04ec-470b-8524-718508882ee5', tool_call_id='call_RQvXyoNDa3CxWF5NqLueiTl0', status='error'),
  AIMessage(content=[], add